# POWER ANALYSIS FOR ONE-SAMPLE T-TEST | R DATA ANALYSIS EXAMPLES

## Examples

Example 1. A company that manufactures light bulbs claims that a particular type of light bulb will last 850 hours on average with standard deviation of 50. A consumer protection group thinks that the manufacturer has overestimated the lifespan of their light bulbs by about 40 hours. How many light bulbs does the consumer protection group have to test in order to prove their point with reasonable confidence?

Example 2. It has been estimated that the average height of American white male adults is 70 inches. It has also been postulated that there is a positive correlation between height and intelligence. If this is true, then the average height of a white male graduate students on campus should be greater than the average height of American white male adults in general. You want to test this theory out by random sampling a small group of white male graduate students. But you need to know how small the group can be or how few people that you need to measure such that you can still prove your point.

## Prelude to The Power Analysis

For the power analysis below, we are going to focus on Example 1 testing the average lifespan of a light bulb. Our first goal is to figure out the number of light bulbs that need to be tested.  That is, we will determine the sample size for a given a significance level and power. Next, we will reverse the process and determine the power, given the sample size and the significance level.

We know so far that the manufacturer claims that the average lifespan of the light bulb is 850 with the standard deviation of 50, and the consumer protection group believes that the manufactory has overestimated by about 40 hours. So in terms of hypotheses, our null hypothesis is $H_0 = 850$ and our alternative hypothesis is $H_a= 810$.

The significance level is the probability of a Type I error, that is the probability of rejecting $H_0$ when it is actually true. We will set it at the .05 level. The power of the test against $H_a$ is the probability of that the test rejects $H_0$. We will set it at .90 level.

We are almost ready for our power analysis. But let’s talk about the standard deviation a little bit. Intuitively, the number of light bulbs we need to test depends on the variability of the lifespan of these light bulbs. Take an extreme case where all the light bulbs have exactly the same lifespan. Then we just need to check a single light bulb to prove our point. Of course, this will never happen. On the other hand, suppose that some light bulbs last for 1000 hours and some only last 500 hours. We will have to select quite a few of light bulbs to cover all the ground. Therefore, the standard deviation for the distribution of the lifespan of the light bulbs will play an important role in determining the sample size.

## Power Analysis

In R, it is fairly straightforward to perform a power analysis for comparing means. For example, we can use R’s `pwr.t.test` function for our calculation as shown below. First, we specify the two means, the mean for the null hypothesis and the mean for the alternative hypothesis divided by the standard deviation for the population. We also need to set the alpha level (.05 for our example), type equal to one.sample and alternative equal to two.sided (two-tail).

In [1]:
from ipywidgets import *
from statsmodels.stats.power import tt_solve_power

In [2]:
def f(H0,Ha,stddev,nobs,alpha,power,alternative):
    effect_size = (H0-Ha)/stddev
    varlist = [nobs,effect_size,alpha,power,alternative]
    varnames = ['n','d','alpha','power','alternative']
    print("One-sample t test power calculation\n")
    for i in range(len(varlist)):
        if varlist[i] != None:
            if varlist[i] == nobs or varlist[i] == alternative:
                print("{0} = {1}".format(varnames[i].rjust(15),str(varlist[i])))
            else:
                print("{0} = {1:.3f}".format(varnames[i].rjust(15),varlist[i]))
        else:
            print("{0} = {1:.3f}".format(varnames[i].rjust(15),
                  tt_solve_power(effect_size,nobs,alpha,power,alternative)
                 ))

In [3]:
interact(f,
         H0=fixed(850),
         Ha=fixed(810),
         stddev=IntSlider(min=1,max=50,value=50),
         nobs=fixed(None),
         alpha=fixed(0.05),
         power=FloatSlider(min=0.5,max=0.99,value=0.9,step=0.01),
         alternative=fixed("two-sided"));

interactive(children=(IntSlider(value=50, description='stddev', max=50, min=1), FloatSlider(value=0.9, descrip…

The result tells us that we need a sample size at least 19 light bulbs to reject $H_0$ under the alternative hypothesis $H_a$ to have a power of 0.9.

Next, suppose we have a sample of size 10, how much power do we have keeping all of the other numbers the same? We can use the same program, `sampsi`, to calculate it.

In [4]:
interact(f,
         H0=fixed(850),
         Ha=fixed(810),
         stddev=IntSlider(min=1,max=50,value=50),
         nobs=IntSlider(min=5,max=20,value=10),
         alpha=fixed(0.05),
         power=fixed(None),
         alternative=fixed("two-sided"));

interactive(children=(IntSlider(value=50, description='stddev', max=50, min=1), IntSlider(value=10, descriptio…

You can see that the power is about .616 for a sample size of 10. What then is the power for sample size of 15?

So now the power is about .82. You could also do it again to find out the power for a sample size of 20. You’ll probably expect that the power will be greater.

We can also expect that if we specified a lower power or the standard deviation is smaller, then the sample size should also be smaller. We can experiment with different values of power and standard deviation as shown below.

In [16]:
interact(f,
         H0=fixed(850),
         Ha=fixed(810),
         stddev=IntSlider(min=1,max=50,value=50),
         nobs=fixed(None),
         alpha=fixed(0.05),
         power=FloatSlider(min=0.5,max=0.99,value=0.8,step=0.01),
         alternative=fixed("two-sided"));

interactive(children=(IntSlider(value=50, description='stddev', max=50, min=1), FloatSlider(value=0.8, descrip…

If the standard deviation is lower, then the sample size should also go down, as we discussed before.

Check it for stddev=30 and see that the sample size goes down.

## Discussion

There is another technical assumption, the normality assumption. If the variable is not normally distributed, a small sample size usually will not have the power indicated in the results, because those results are calculated using the common method based on the normality assumption. It might not even be a good idea to do a t-test on such a small sample to begin with if the normality assumption is in question.

Here is another technical point. What we really need to know is the difference between the two means, not the individual values. In fact, what really matters is the difference of the means over the standard deviation. We call this the effect size. For example, we would get the same power if we subtracted 800 from each mean, changing 850 to 50 and 810 to 10.

In [11]:
pwr.t.test(d=(50-10)/50,power=0.9,sig.level=0.05,type="one.sample",alternative="two.sided")


     One-sample t test power calculation 

              n = 18.44624
              d = 0.8
      sig.level = 0.05
          power = 0.9
    alternative = two.sided


If we standardize our variable, we can calculate the means in terms of change in standard deviations.

In [12]:
pwr.t.test(d=(1-.2),power=0.9,sig.level=0.05,type="one.sample",alternative="two.sided")


     One-sample t test power calculation 

              n = 18.44624
              d = 0.8
      sig.level = 0.05
          power = 0.9
    alternative = two.sided


It is usually not an easy task to determine the "true" effect size. We make our best guess based upon the existing literature or a pilot study. A good estimate of the effect size is the key to a successful power analysis.

## See Also

* Related R Commands
 * `pwr.t.test` — Sample size and power determination.
* References
 * D. Moore and G. McCabe, Introduction to the Practice of Statistics, Third Edition, Section 6.4.